In [3]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from pprint import pprint


In [4]:
cases_311_df = pd.read_csv("311_Cases.csv", low_memory=False)
cases_311_df

,version https://git-lfs.github.com/spec/v1
0,oid sha256:1de5d0b4618f18706ad1f802f522683f460...
1,size 1776525047


In [5]:
cases_311_df.count()


version https://git-lfs.github.com/spec/v1    2
dtype: int64

In [6]:
cases_311_df['Neighborhoods'].head()

KeyError: 'Neighborhoods'

In [5]:
cases_311_df['DELETE - Zip Codes']

0              55
1             309
2              55
3          28,861
4             309
            ...  
4139964       309
4139965    28,856
4139966        64
4139967       308
4139968        56
Name: DELETE - Zip Codes, Length: 4139969, dtype: object

In [6]:
cases_311_df['Source'].value_counts()

Phone                1887797
Mobile/Open311       1461762
Web                   617848
Integrated Agency     131650
Twitter                33832
Other Department        6025
Email                   1042
Mail                      13
Name: Source, dtype: int64

In [7]:
cases_311_df['Category'].value_counts()

Street and Sidewalk Cleaning              1405419
Graffiti                                   550660
Encampments                                254585
Abandoned Vehicle                          241026
MUNI Feedback                              233507
                                           ...   
General Request - RISK MANAGEMENT               3
General Request - JUVENILE PROBATION            3
General Request - BOARD OF APPEALS              2
General Request - REDEVELOPMENT AGENCY          1
General Request - MUSEUMS                       1
Name: Category, Length: 103, dtype: int64

In [8]:
cases_311_df['Request Type'].value_counts()

Bulky Items                                                                      555602
General Cleaning                                                                 498605
Encampment Reports                                                               223336
request_for_service                                                              171162
Human or Animal Waste                                                            166971
                                                                                  ...  
Construction Zone Tow-away Permits for Triton Towers                                  1
Construction Zone Tow-away Permits for Conquest Construction                          1
Construction Zone Tow-away Permits for P Jordan                                       1
Construction Zone Tow-away Permits for Manuel Brothers/Golden State Utilities         1
Construction Zone Tow-away Permits for THE FAIRMONT SAN FRANCISCO HOTEL               1
Name: Request Type, Length: 1234

In [9]:
cases_311_df['Neighborhood'].value_counts()


Mission                     397138
South of Market             305256
Tenderloin                  129457
Outer Sunset                119532
Mission Dolores             107121
                             ...  
Lincoln Park / Ft. Miley      1430
Monterey Heights              1195
Treasure Island               1097
Yerba Buena Island             164
8                                1
Name: Neighborhood, Length: 118, dtype: int64

In [12]:
nbh_vc = cases_311_df['Neighborhood'].value_counts()
print(type(nbh_vc))
nbh_vc.index

<class 'pandas.core.series.Series'>


Index(['Mission', 'South of Market', 'Tenderloin', 'Outer Sunset',
       'Mission Dolores', 'Inner Richmond', 'Potrero Hill', 'Outer Richmond',
       'Bernal Heights', 'Lower Nob Hill',
       ...
       'St. Francis Wood', 'Balboa Terrace', 'Candlestick Point SRA',
       'Presidio National Park', 'Westwood Highlands',
       'Lincoln Park / Ft. Miley', 'Monterey Heights', 'Treasure Island',
       'Yerba Buena Island', '8'],
      dtype='object', length=118)

In [13]:
# print out 118 Neighborhoods
for nbh in nbh_vc.index:
    print(nbh)

Mission
South of Market
Tenderloin
Outer Sunset
Mission Dolores
Inner Richmond
Potrero Hill
Outer Richmond
Bernal Heights
Lower Nob Hill
Portola
Bayview
Civic Center
Excelsior
Chinatown
Nob Hill
Duboce Triangle
North Beach
Western Addition
Lower Haight
Pacific Heights
Russian Hill
Inner Sunset
Financial District
Cathedral Hill
Parkside
Downtown / Union Square
Castro
Panhandle
Haight Ashbury
Mission Terrace
Hayes Valley
Marina
Noe Valley
Dolores Heights
Buena Vista
Silver Terrace
Polk Gulch
Golden Gate Park
Visitacion Valley
Cayuga
Lower Pacific Heights
Lone Mountain
Showplace Square
Oceanview
Eureka Valley
Golden Gate Heights
Ingleside
Crocker Amazon
Bret Harte
Union Street
Holly Park
Central Waterfront
Sunnyside
Hunters Point
Sunnydale
South Beach
Mission Bay
Rincon Hill
Merced Heights
Outer Mission
Produce Market
Alamo Square
Cole Valley
Mint Hill
Laurel Heights / Jordan Park
Apparel City
Peralta Heights
Dogpatch
Glen Park
Japantown
Northern Waterfront
Miraloma Park
Upper Market
Univ

In [ ]:
from config import api_key
c = Census(api_key, year=2017)
c = Census(api_key)

In [16]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

In [17]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.tail(100)

33120


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
33020,99701,17608.0,32.2,58275.0,30330.0,2481.0,14.090186
33021,99702,3025.0,23.8,57464.0,26631.0,115.0,3.801653
33022,99703,8415.0,22.9,54526.0,22746.0,448.0,5.323827
33023,99704,30.0,20.7,-666666666.0,25570.0,0.0,0.000000
33024,99705,22804.0,32.0,81045.0,34618.0,1430.0,6.270830
...,...,...,...,...,...,...,...
33115,99923,0.0,-666666666.0,-666666666.0,-666666666.0,0.0,NaN
33116,99925,901.0,45.1,48646.0,24783.0,210.0,23.307436
33117,99926,1684.0,32.8,57969.0,22927.0,224.0,13.301663
33118,99927,59.0,49.8,17981.0,11959.0,49.0,83.050847


In [18]:
sf_df = pd.DataFrame()
sf_df["Zipcode"]  = [94127, 94105,94123,94130,94131,94114,94129,94116,94117,94121,
                    94118,94107,94122,94112,94111,94132,94115,94134,
                    94110,94109,94133,94124,94108,94103,94102,94104,94128]
sf_df["Zipcode"] = sf_df["Zipcode"].astype(str)
summary_df = pd.merge(census_pd, sf_df, on = ['Zipcode'])
summary_df.to_csv("summary_data.csv", encoding="utf-8", index=False)
summary_df

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,94102,30140.0,41.5,33552.0,43488.0,6957.0,23.082283
1,94103,26990.0,38.8,49052.0,55807.0,5870.0,21.748796
2,94104,436.0,51.3,48429.0,64267.0,84.0,19.266055
3,94105,7675.0,37.6,199364.0,154723.0,733.0,9.550489
4,94107,29920.0,35.9,143467.0,87735.0,3391.0,11.333556
5,94108,14856.0,42.8,55341.0,51359.0,3178.0,21.392030
6,94109,56587.0,36.9,79979.0,75460.0,6718.0,11.871985
7,94110,73737.0,36.6,109747.0,59660.0,7321.0,9.928530
8,94111,3356.0,46.1,99464.0,88610.0,661.0,19.696067
9,94112,85373.0,41.4,82692.0,31905.0,7750.0,9.077811
